In [1]:
# Import libraries
import sys

sys.path.append("..")

from monai.config import print_config
from src.preprocessing import get_transforms, get_datasets, get_dataloaders

import torch
import matplotlib.pyplot as plt
from src.config import config
from src.model import SegFormer3D

print_config()

# Set the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ldeben20 (luciano-deben). Use `wandb login --relogin` to force relogin


MONAI version: 1.3.0
Numpy version: 1.26.4
Pytorch version: 2.2.1
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 865972f7a791bf7b42efbcd87c8402bd865b329e
MONAI __file__: c:\Users\<username>\AppData\Local\anaconda3\envs\torch-gpu\Lib\site-packages\monai\__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
ITK version: 5.3.0
Nibabel version: 5.2.1
scikit-image version: NOT INSTALLED or UNKNOWN VERSION.
scipy version: 1.12.0
Pillow version: 10.2.0
Tensorboard version: 2.16.2
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: 0.17.1
tqdm version: 4.66.2
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.9.8
pandas version: 2.2.1
einops version: 0.7.0
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: NOT INSTALLED or UNKNOWN VERSION.
pynrrd version: NOT INSTALLED or UNKNOWN VERSION.
clearml version: NOT INSTALLED or UNKNOWN VERSION.

For details about ins

In [2]:
# Get the transforms
transform = get_transforms(
    resize_shape=config["resize_shape"], contrast_value=config["contrast_value"]
)

# Get the datasets
train_dataset, val_dataset = get_datasets(
    root_dir="../data",
    collection="HCC-TACE-Seg",
    transform=transform,
    download=True,
    download_len=2,
    seg_type="SEG",
    val_frac=config["val_frac"],
    seed=config["seed"],
)

# Get the dataloaders
train_loader, val_loader = get_dataloaders(
    train_dataset,
    val_dataset,
    batch_size=config["batch_size"],
    num_workers=config["num_workers"],
)

2024-05-09 10:56:43,732 - INFO - Expected md5 is None, skip md5 check for file ..\data\HCC-TACE-Seg\1.2.276.0.7230010.3.1.3.8323329.41.1604860085.518229.zip.
2024-05-09 10:56:43,734 - INFO - File exists: ..\data\HCC-TACE-Seg\1.2.276.0.7230010.3.1.3.8323329.41.1604860085.518229.zip, skipped downloading.
2024-05-09 10:56:43,736 - INFO - Writing into directory: ..\data\HCC-TACE-Seg\raw\1.2.276.0.7230010.3.1.3.8323329.41.1604860085.518229.
2024-05-09 10:56:44,459 - INFO - Expected md5 is None, skip md5 check for file ..\data\HCC-TACE-Seg\1.3.6.1.4.1.14519.5.2.1.1706.8374.172517341095680731665822868712.zip.
2024-05-09 10:56:44,460 - INFO - File exists: ..\data\HCC-TACE-Seg\1.3.6.1.4.1.14519.5.2.1.1706.8374.172517341095680731665822868712.zip, skipped downloading.
2024-05-09 10:56:44,461 - INFO - Writing into directory: ..\data\HCC-TACE-Seg\HCC_017\300\image.
2024-05-09 10:56:44,806 - INFO - Expected md5 is None, skip md5 check for file ..\data\HCC-TACE-Seg\1.2.276.0.7230010.3.1.3.8323329.208

In [3]:
# Sample a batch of data from the dataloader
batch = next(iter(train_loader))

# Separate the image and segmentation from the batch
image, seg = batch["image"], batch["seg"]

print(image.shape, seg.shape)

torch.Size([1, 1, 256, 256, 48]) torch.Size([1, 5, 256, 256, 48])


In [4]:
# Evaluate the model
model = SegFormer3D()
model.load_state_dict(torch.load("../models/model.pth"))
model.eval()

# Move the model to the device
model = model.to(device)

In [5]:
# Set a evaluation script
outputs = model(image.to(device))
max_values, prediction = torch.max(outputs, 1)
print("Prediction Values:", prediction.unique())
print("Prediction Shape:", prediction.shape)

Prediction Values: metatensor([0, 1, 2, 3, 4], device='cuda:0')
Prediction Shape: torch.Size([1, 256, 256, 48])


In [6]:
import nibabel as nib
import numpy as np

# Check if the first dimension is 1 before squeezing
if prediction.size(0) == 1:
    prediction = prediction.squeeze(0)

# Convert to numpy array and change dtype to float32
prediction_numpy = prediction.cpu().numpy().astype(np.float32)

new_image = nib.Nifti1Image(prediction_numpy, np.eye(4))
nib.save(new_image, "prediction.nii")